In [1]:
import signal
import requests
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
import concurrent.futures
import threading
import re

In [2]:
# code that lets us shut down if CTRL C is pressed
def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

In [3]:
# class that passes error message, ends the program
class ApiException(Exception):
    pass

In [4]:
#Setting Up global empty dictonaries
CAPM_vals = {}
expected_return = {}

In [35]:
#Creating a session to connect with RIT
API_KEY = {'X-API-Key': 'MCKGCURR'}
shutdown = False
session = requests.Session()
session.headers.update(API_KEY)

In [6]:
#code that gets the current tick
def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('fail - cant get tick')

In [7]:
#code that parses the first and latest news instances for forward market predictions and the risk free rate
#Important: this code only works if the only '%' character is in front of the RISK FREE RATE and the onle '$' character is in front of the forward price suggestions
def get_news(session):
    news = session.get('http://localhost:9999/v1/news')    
    if news.ok:
        newsbook = news.json()
        try:
            for i in range(len(newsbook[-1]['body'])):
                if newsbook[-1]['body'][i] == '%':
                    CAPM_vals['%Rf'] = round(float(newsbook[-1]['body'][i - 4:i])/100, 4)               
            latest_news = newsbook[0]
            if len(newsbook) > 1:
                for j in range(len(latest_news['body']) - 1, 1, -1):    
                    while latest_news['body'][j] != '$':
                        j -= 1
                CAPM_vals['forward'] = float(latest_news['body'][j + 1:-1])
                CAPM_vals['tick_forward'] = int(latest_news['body'][latest_news['body'].index('tick') + 5:latest_news['body'].index(',')])
            return CAPM_vals, len(newsbook)
        except:
            return CAPM_vals, len(newsbook)
    raise ApiException('timeout')

In [8]:
print(session.get('http://localhost:9999/v1/securities').json())

{'code': 'UNAUTHORIZED', 'message': 'Must be logged in.'}


In [40]:
#gets all the price data for all securities        
def pop_prices(session):
    price_act = session.get('http://localhost:9999/v1/securities')
    if price_act.ok:
        prices = price_act.json()
        dic = {}
        for item in prices:
            dic[item['ticker']] = item['last']
        return dic
    raise ApiException('fail - cant get securities')

In [10]:
#Buy or Sell function, put in your own parameters
def buy_or_sell(session, expected_return):
    for i in expected_return.keys():
        if expected_return[i] > 'ENTER YOUR PARAMETER':
            session.post('http://localhost:9999/v1/orders', params = {'ticker': i, 'type': 'ENTER ORDER TYPE', 'quantity': 'ENTER DESIRED QUANTITY', 'action': 'BUY/SELL'})
        elif expected_return[i] < 'ENTER YOUR PARAMETER':
            session.post('http://localhost:9999/v1/orders', params = {'ticker': i, 'type': 'ENTER ORDER TYPE', 'quantity': 'ENTER DESIRED QUANTITY', 'action': 'BUY/SELL'})

In [11]:
#Beta: Apha: Low; GAMMA: High; THETA: Medium
#Volatility: Apha: Low; GAMMA: High; THETA: Medium
buy_order = {}
sell_order = {}

In [12]:
def Take_full_positive(session,position_ticker,hedge_ticker):
    for a in range(10):
        resp = session.get('http://localhost:9999/v1/limits')
        if resp.ok:
            case = resp.json()
        if case[0]['net'] <= 100000:
            session.post('http://localhost:9999/v1/orders', params = {'ticker': position_ticker, 'type': 'MARKET', 'quantity': 10000, 'action': 'BUY'})
            session.post('http://localhost:9999/v1/orders', params = {'ticker': hedge_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'SELL'})
            session.post('http://localhost:9999/v1/orders', params = {'ticker': position_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'BUY'})         

In [13]:
def take_full_positive(session, position_ticker, hedge_ticker):
    def process_iteration(_):
        resp = session.get('http://localhost:9999/v1/limits')
        if resp.ok:
            case = resp.json()
            if case[0]['gross'] <= 250000:
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'MARKET', 'quantity': 10000, 'action': 'BUY'})
                session.post('http://localhost:9999/v1/orders', params={'ticker': hedge_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'SELL'})
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'BUY'})

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks for the first batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the first batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the second batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the second batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the third batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

In [14]:
def take_full_positive_limit(session, position_ticker, hedge_ticker, price_full,price_hedge):
    def process_iteration(_):
        resp = session.get('http://localhost:9999/v1/limits')
        if resp.ok:
            case = resp.json()
            if case[0]['gross'] <= 250000:
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'MARKET', 'quantity': 10000, 'action': 'BUY','price' :price_full})
                session.post('http://localhost:9999/v1/orders', params={'ticker': hedge_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'SELL','price' :price_hedge})
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'BUY','price' :price_full})

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks for the first batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the first batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the second batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the second batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the third batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

In [15]:
def Take_full_negative(session,position_ticker,hedge_ticker):
    for a in range(10):
        resp = session.get('http://localhost:9999/v1/limits')
        if resp.ok:
            case = resp.json()
        if case[0]['net'] <= 100000:
            session.post('http://localhost:9999/v1/orders', params = {'ticker': position_ticker, 'type': 'MARKET', 'quantity': 10000, 'action': 'SELL'})
            session.post('http://localhost:9999/v1/orders', params = {'ticker': hedge_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'BUY'})
            session.post('http://localhost:9999/v1/orders', params = {'ticker': position_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'SELL'})  

In [16]:
def getTotalPosition(session, ticker):
    response = session.get("http://localhost:9999/v1/securities?ticker=" + ticker)
    content = response.json()
    return content[0]['position']

def closePosition(session, ticker):
    position = getTotalPosition(session, ticker)
    sendMarketOrderUnderVolumeConstraint(session, position, ticker)

def sendMarketOrderUnderVolumeConstraint(session, quantity, ticker):
    if quantity > 10000:
        quantity = 10000
    elif quantity < -10000:
        quantity = -10000
    session.post('http://localhost:9999/v1/orders', params={'ticker': ticker, 'type': 'MARKET', 'quantity': quantity, 'action': 'SELL'})
    return quantity

In [17]:
def Get_out_postive(session, position_ticker,hedge_ticker):
    for 

SyntaxError: invalid syntax (3780178049.py, line 2)

In [18]:
def take_full_negative(session, position_ticker, hedge_ticker):
    def process_iteration(_):
        resp = session.get('http://localhost:9999/v1/limits')
        if resp.ok:
            case = resp.json()
            if case[0]['gross'] <= 250000:
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'MARKET', 'quantity': 10000, 'action': 'SELL'})
                session.post('http://localhost:9999/v1/orders', params={'ticker': hedge_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'BUY'})
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'SELL'})

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks for the first batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the first batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the second batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the second batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the third batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

In [19]:
def take_full_negative_limit(session, position_ticker, hedge_ticker, price_full, price_hedge):
    def process_iteration(_):
        resp = session.get('http://localhost:9999/v1/limits')
        if resp.ok:
            case = resp.json()
            if case[0]['gross'] <= 250000:
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'LIMIT', 'quantity': 10000, 'action': 'SELL', 'price' :price_full})
                session.post('http://localhost:9999/v1/orders', params={'ticker': hedge_ticker, 'type': 'LIMIT', 'quantity': 7500, 'action': 'BUY', 'price' :price_hedge})
                session.post('http://localhost:9999/v1/orders', params={'ticker': position_ticker, 'type': 'LIMIT', 'quantity': 7500, 'action': 'SELL', 'price' :price_full})

    # Submit all tasks at once
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks for the first batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the first batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the second batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for the second batch to complete
        concurrent.futures.wait(futures)
        sleep(0.8)
        # Submit tasks for the third batch
        futures = [executor.submit(process_iteration, None) for _ in range(10)]
        
        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

In [20]:
def calculate_max_min(capm_df):
    # Calculate the total average of all CAPM values of the entire case
    total_avg_alpha = capm_df['ALPHA'].mean()
    total_avg_gamma = capm_df['GAMMA'].mean()
    total_avg_theta = capm_df['THETA'].mean()
    

    # Calculate the average of last 18 values stored in capm_df
    last_18_avg_alpha = capm_df.head(6)['ALPHA'].mean()
    last_18_avg_gamma = capm_df.head(6)['GAMMA'].mean()
    last_18_avg_theta = capm_df.head(6)['THETA'].mean()
    
    dic = {}

    # Calculate the absolute difference between the total average and the average of last 18 values
    dic['ALPHA'] = (last_18_avg_alpha)* capm_df['Price-ALPHA'].iloc[0]#- total_avg_alpha*0.8
    dic['GAMMA'] = (last_18_avg_gamma)* capm_df['Price-GAMMA'].iloc[0]#- total_avg_gamma*0.8
    dic['THETA'] = (last_18_avg_theta)* capm_df['Price-THETA'].iloc[0]#- total_avg_theta*0.8
    
    # Calculate the Max and Min values
    max_value = max(dic, key=dic.get)
    min_value = min(dic, key=dic.get)     

    return max_value, min_value, dic[max_value], dic[min_value]

In [21]:
def max_min_check_4(capm_df, current_price):
    

IndentationError: expected an indented block (662853318.py, line 2)

In [29]:
#Create trade signal to make, sell, buy or not doing anything decisions
def decision_maker():
    

SyntaxError: incomplete input (3993423595.py, line 3)

In [22]:
def get_out_of_position():
    resp = session.get('http://localhost:9999/v1/securities')
    if resp.ok:
        case = resp.json()
    while case[1]['position'] != 0:
        quantity = sendMarketOrderUnderVolumeConstraint(session, case[1]['position'], case[1]['ticker'])
        case[1]['position'] = case[1]['position'] - quantity
    while case[2]['position'] != 0:
        quantity = sendMarketOrderUnderVolumeConstraint(session, case[2]['position'], case[2]['ticker'])
        case[2]['position'] = case[2]['position'] - quantity
    while case[3]['position'] != 0:
        quantity = sendMarketOrderUnderVolumeConstraint(session, case[3]['position'], case[3]['ticker'])
        case[3]['position'] = case[3]['position'] - quantity

In [23]:
def sendLimitOrderUnderVolumeConstraint(session, quantity, ticker,price):
    if quantity > 10000:
        quantity = 10000
    elif quantity < -10000:
        quantity = -10000
    session.post('http://localhost:9999/v1/orders', params={'ticker': ticker, 'type': 'LIMIT', 'quantity': quantity, 'action': 'SELL', 'price':price})
    return quantity

In [24]:
def get_out_of_position_limit():
    resp = session.get('http://localhost:9999/v1/securities')
    if resp.ok:
        case = resp.json()
    position_1 = case[1]['position']
    position_2 = case[2]['position']
    position_3 = case[3]['position']   
    while position_1 != 0:
        quantity = sendLimitOrderUnderVolumeConstraint(session, position_1, case[1]['ticker'], case[1]['last'])
        position_1 = position_1 - quantity
    while position_2 != 0:
        quantity = sendLimitOrderUnderVolumeConstraint(session, position_2, case[2]['ticker'], case[2]['last'])
        position_2 = position_2 - quantity
    while position_3 != 0:
        quantity = sendLimitOrderUnderVolumeConstraint(session, position_3, case[3]['ticker'], case[3]['last'])
        position_3 = position_3 - quantity

In [25]:
def get_out_gracefully(full_in, hedge_by, price_full_in, position_type):
    resp = session.get('http://localhost:9999/v1/securities')
    current_situation = 'Bad'
    if resp.ok:
        case = resp.json()
    for item in case:
        # try:
        if item['ticker'] == full_in:     
            if float(item['last']) > float(price_full_in) and position_type == 'Positive':
                while item['position'] != 0:
                    while item['position'] != 0:
                        quantity = sendMarketOrderUnderVolumeConstraint(session, item['position'], full_in)
                        item['position'] = item['position'] - quantity
                        current_situation = 'Good'
            elif float(item['last']) < float(price_full_in) and position_type == 'Positive':
                while item['position'] != 0:
                    while item['position'] != 0:
                        quantity = sendLimitOrderUnderVolumeConstraint(session, item['position'], full_in, float(price_full_in) + 0.5)
                        item['position'] = item['position'] - quantity
                        current_situation = 'Bad'
            elif float(item['last']) < float(price_full_in) and position_type == 'Negative':
                while item['position'] != 0:
                    while item['position'] != 0:
                        quantity = sendMarketOrderUnderVolumeConstraint(session, item['position'], full_in)
                        item['position'] = item['position'] - quantity
                        current_situation = 'Good'
            elif float(item['last']) > float(price_full_in) and position_type == 'Negative':
                while item['position'] != 0:
                    while item['position'] != 0:
                        quantity = sendLimitOrderUnderVolumeConstraint(session, item['position'], full_in, float(price_full_in) - 0.5)
                        item['position'] = item['position'] - quantity
                        current_situation = 'Bad'
            else:
                get_out_of_position()
        else:
            pass                                                         
        # except:
        #     print("Inexcept")
        #     get_out_of_position()
    for item in case:
        if item['ticker'] == hedge_by and current_situation == 'Good':
            get_out_of_position()                                

In [26]:
def try_main():
    with requests.Session() as session:
        session.Session() as headers
        ritm

SyntaxError: invalid syntax (667007828.py, line 3)

In [27]:
def main():
    with requests.Session() as session:
        session.headers.update(API_KEY)
        ritm = pd.DataFrame(columns= ['RITM','BID', 'ASK', 'LAST', '%Rm'])
        alpha = pd.DataFrame(columns= ['ALPHA','BID', 'ASK', 'LAST', '%Ri', '%Rm'])
        gamma = pd.DataFrame(columns= ['GAMMA','BID', 'ASK', 'LAST', '%Ri', '%Rm'])
        theta = pd.DataFrame(columns= ['THETA','BID', 'ASK', 'LAST', '%Ri', '%Rm'])
        beta_df = pd.DataFrame(columns= ['ALPHA', 'GAMMA', 'THETA'])
        # try:
        #     capm_df = pd.read_csv('capm_df.csv')
        # except FileNotFoundError:
        capm_df = pd.DataFrame(columns=['ALPHA', 'GAMMA', 'THETA'])
        max_arr = [None]*15
        min_arr = [None]*15
        index_arr = 0
        current_position = False
        positve_position = False
        negative_position = False
        take_full_in = 'ALPHA'
        hedge_by = 'THETA'
        buy_price = 0
        
        execution_df = pd.DataFrame(columns = ['ALPHA_Buy_price','GAMMA_Buy_price', 'THETA_Buy_price', 'Buy_decision', 'Sell_decision','positive_position','Negative_position','Position','ALPHA - Beta', 'GAMMA - Beta', 'THETA - Beta'])            
    
        while get_tick(session) < 600 and not shutdown:
            #update the forward market price and rf rate
            temp ,news_len = get_news(session)
            
            ##update RITM bid-ask dataframe
            pdt_RITM = pd.DataFrame(pop_prices(session)[0])
            ritmp = pd.DataFrame({'RITM': '', 'BID': pdt_RITM['bid'],'ASK': pdt_RITM['ask'], 'LAST': pdt_RITM['last'], '%Rm': ''})
            if ritm['BID'].empty or ritmp['LAST'].iloc[0] != ritm['LAST'].iloc[0]:
                ritm = pd.concat([ritmp, ritm.loc[:]]).reset_index(drop=True)
                ritm['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
                if ritm.shape[0] >= 61:
                    ritm = ritm.iloc[:60]
            
            #generate expected market return paramter
            if 'forward' in CAPM_vals.keys():
                CAPM_vals['%RM'] = (CAPM_vals['forward']-ritm['LAST'].iloc[0])/ritm['LAST'].iloc[0]
            else:
                CAPM_vals['%RM'] = ''
                
                
              
            ##update ALPHA bid-ask dataframe
            pdt_ALPHA = pd.DataFrame(pop_prices(session)[1])
            alphap = pd.DataFrame({'ALPHA': '', 'BID': pdt_ALPHA['bid'],'ASK': pdt_ALPHA['ask'], 'LAST': pdt_ALPHA['last'], '%Ri': '', '%Rm': ''})
            if alpha['BID'].empty or alphap['LAST'].iloc[0] != alpha['LAST'].iloc[0]:
                alpha = pd.concat([alphap, alpha.loc[:]]).reset_index(drop=True)
                alpha['%Ri'] = (alpha['LAST']/alpha['LAST'].shift(-1)) - 1
                alpha['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
                if alpha.shape[0] >= 61:
                    alpha = alpha.iloc[:60]
              
            ##update GAMMA bid-ask dataframe
            pdt_GAMMA = pd.DataFrame(pop_prices(session)[2])
            gammap = pd.DataFrame({'GAMMA': '', 'BID': pdt_GAMMA['bid'],'ASK': pdt_GAMMA['ask'], 'LAST': pdt_GAMMA['last'], '%Ri': '', '%Rm': ''})
            if gamma['BID'].empty or gammap['LAST'].iloc[0] != gamma['LAST'].iloc[0]:
                gamma = pd.concat([gammap, gamma.loc[:]]).reset_index(drop=True)
                gamma['%Ri'] = (gamma['LAST']/gamma['LAST'].shift(-1)) - 1
                gamma['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
                if gamma.shape[0] >= 61:
                    gamma = gamma.iloc[:60]            
            
            ##update THETA bid-ask dataframe
            pdt_THETA = pd.DataFrame(pop_prices(session)[3])
            thetap = pd.DataFrame({'THETA': '', 'BID': pdt_THETA['bid'],'ASK': pdt_THETA['ask'], 'LAST': pdt_THETA['last'], '%Ri': '', '%Rm': ''})
            if theta['BID'].empty or thetap['LAST'].iloc[0] != theta['LAST'].iloc[0]:     
                theta = pd.concat([thetap, theta.loc[:]]).reset_index(drop=True)
                theta['%Ri'] = (theta['LAST']/theta['LAST'].shift(-1)) - 1
                theta['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
                if theta.shape[0] >= 61:
                    theta = theta.iloc[:60]
            
            beta_alpha = (alpha['%Ri'].cov(ritm['%Rm']))/(ritm['%Rm'].var())
            beta_gamma = (gamma['%Ri'].cov(ritm['%Rm']))/(ritm['%Rm'].var())
            beta_theta = (theta['%Ri'].cov(ritm['%Rm']))/(ritm['%Rm'].var())
            
            
            beta_p = pd.DataFrame({'ALPHA': beta_alpha, 'GAMMA': beta_gamma, 'THETA': beta_theta}, index = [0])
            if beta_df['ALPHA'].empty or beta_p['ALPHA'].iloc[0] != beta_df['ALPHA'].iloc[0]:     
                beta_df = pd.concat([beta_p, beta_df.loc[:]]).reset_index(drop=True)
                if beta_df.shape[0] >= 41:
                    beta_df = beta_df.iloc[:40]
             
            
            CAPM_vals['Beta - ALPHA'] = beta_alpha
            CAPM_vals['Beta - GAMMA'] = beta_gamma
            CAPM_vals['Beta - THETA'] = beta_theta
            
            if CAPM_vals['%RM'] != '':
                er_alpha = CAPM_vals['%Rf'] + CAPM_vals['Beta - ALPHA'] * (CAPM_vals['%RM'] - CAPM_vals['%Rf'])
                er_gamma = CAPM_vals['%Rf'] + CAPM_vals['Beta - GAMMA'] * (CAPM_vals['%RM'] - CAPM_vals['%Rf'])
                er_theta = CAPM_vals['%Rf'] + CAPM_vals['Beta - THETA'] * (CAPM_vals['%RM'] - CAPM_vals['%Rf'])
            else:
                er_alpha = 'Wait for market forward price'
                er_gamma = 'Wait for market forward price'
                er_theta = 'Wait for market forward price'
                
            expected_return['ALPHA'] = er_alpha
            expected_return['GAMMA'] = er_gamma
            expected_return['THETA'] = er_theta
            
            
            current_tick = get_tick(session)
            # previous_tick = ''
            # if current_tick != previous_tick:
            try:
                news = session.get(f"http://localhost:9999/v1/news")
                news = news.json()[0]['body']
            except:
                news = "Not able to read news or no news yet."    
            capm_df_p = pd.DataFrame({'Tick':current_tick,'ALPHA': er_alpha, 'GAMMA': er_gamma,'THETA': er_theta, 'Price-ALPHA':pdt_ALPHA['last'], 'Price-GAMMA':pdt_GAMMA['last'],'Price-THETA':pdt_THETA['last'], 'Price-RITM':pdt_RITM['last'], 'Latest-News': news}, index = [0])
            if capm_df['ALPHA'].empty or (capm_df_p['GAMMA'].iloc[0] != capm_df['GAMMA'].iloc[0]) :
                capm_df = pd.concat([ capm_df_p ,capm_df.loc[:]]).reset_index(drop=True)
                if capm_df.shape[0] >= 4001:
                    capm_df = capm_df.iloc[:4000]
            # previous_tick = current_tick       
                
            #capm_df.to_csv(f'capm_df_with_execution_9.csv', index=False)
                
            
            max_arr[index_arr], min_arr[index_arr], expected_return_max, expected_return_min = calculate_max_min(capm_df)
            
            index_arr += 1
            
            if index_arr == 15:
                index_arr = 0
            
            print(CAPM_vals['tick_forward']- current_tick)
            print(max_arr,min_arr)
            
            # if None not in max_arr and None not in min_arr and (CAPM_vals['tick_forward']- current_tick == 7) and news_len > 1:
            #     dic = {'ALPHA':er_alpha,'GAMMA': er_gamma, 'THETA':er_theta}
            #     max_value = max(dic, key=dic.get)
            #     min_value = min(dic, key=dic.get)
            #     if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0:
            #         take_full_positive(session, max_value, 'THETA')
            #     else:
            #         take_full_negative(session, min_value, 'THETA')    
                    
            if (CAPM_vals['tick_forward']- current_tick == 9):
                dic_price = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}
                tree_check_decision = CAPM_vals['forward']-ritm.head(6)['LAST'].mean()
            else:
                price_current = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}    
            
            if None not in max_arr and None not in min_arr and (CAPM_vals['tick_forward']- current_tick == 8) and news_len > 1:
                take_full_in = max(max_arr)
                beta_full_in = beta_df.head(4)[take_full_in].mean()
                # if beta_full_in < 0:
                #     opposite_position = True
                # else:
                opposite_position = False    
                
                # if take_full_in:
                #     if CAPM_vals['Beta - ALPHA'] > CAPM_vals['Beta - GAMMA']:
                #         if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0:
                #             take_full_in = 'ALPHA'
                #         else:
                #             take_full_in = 'GAMMA'    
                #     else:
                #         if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0:
                #             take_full_in = 'GAMMA'
                #         else:
                #             take_full_in = 'ALPHA'        
                             
                                
                hedge_by =  max(min_arr)
                dic_beta = {'ALPHA': CAPM_vals['Beta - ALPHA'], 'GAMMA': CAPM_vals['Beta - GAMMA'], 'THETA': CAPM_vals['Beta - THETA']}
                
                dic_prcie = dic_price
                
                print(take_full_in)
                print(hedge_by)
                # if abs(CAPM_vals['forward']-ritm['LAST'].iloc[0])>70:
                # if CAPM_vals['tick_forward']- current_tick < 2 and not current_position:
                #     if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0 and not positve_position:
                #         take_full_positive(session, take_full_in, hedge_by)
                # elif CAPM_vals['tick_forward']- current_tick < 0 and current_position:
                #     if positve_position:
                #         take_full_negative(session, take_full_in, hedge_by)
                #     else:
                #         take_full_positive(session, take_full_in, hedge_by)                
                # elif CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0 and not positve_position:
                #     take_full_positive(session, take_full_in, hedge_by)
                #     if negative_position:
                #         current_position = False
                #         take_full_positive(session, take_full_in, hedge_by)
                #     else:
                #         current_position = True
                #     positve_position = True
                #     negative_position = False
                # elif CAPM_vals['forward']-ritm['LAST'].iloc[0] < 0 and not negative_position:
                #     take_full_negative(session, take_full_in, hedge_by)
                #     if positve_position:
                #         current_position = False
                #         take_full_negative(session, take_full_in, hedge_by) 
                #     else:
                #         current_position = True    
                #     positve_position = False
                #     negative_position = True
            # else:
                # if CAPM_vals['tick_forward']- current_tick < 2 and not current_position:
                #     if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0 and not positve_position:
                #         take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
                # elif CAPM_vals['tick_forward']- current_tick < 0 and current_position:
                #     if positve_position:
                #         take_full_negative_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
                #     else:
                #         take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by]) 
                buy_price = dic_price[take_full_in]
                if not opposite_position:              
                    if tree_check_decision > 0 and not positve_position:
                        #take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
                        thread_neg = threading.Thread(target=take_full_positive_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in], dic_price[hedge_by]))
                        thread_neg.start()
                        if negative_position:
                            current_position = False
                            take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
                        else:
                            current_position = True
                        positve_position = True
                        negative_position = False
                    elif tree_check_decision < 0 and not negative_position:
                        dic_price = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}
                        thread_pos = threading.Thread(target = take_full_negative, args=(session, take_full_in, hedge_by))
                        thread_pos.start()
                        if positve_position:
                            current_position = False
                            thread_pos = threading.Thread(target = take_full_negative, args=(session, take_full_in, hedge_by))
                            thread_pos.start() 
                        else:
                            current_position = True    
                        positve_position = False
                        negative_position = True    
                    execution_df_p = pd.DataFrame({'tick': current_tick,'ALPHA_Buy_price':dic_prcie['ALPHA'],'GAMMA_Buy_price': dic_prcie['GAMMA'], 'THETA_Buy_price': dic_prcie['THETA'],'Buy_decision': take_full_in, 'Sell_decision':hedge_by,'positive_position':positve_position,'Negative_position':negative_position,'Position':current_position   ,'ALPHA - Beta': CAPM_vals['Beta - ALPHA'], 'GAMMA - Beta': CAPM_vals['Beta - GAMMA'], 'THETA - Beta': CAPM_vals['Beta - THETA']})
                    if execution_df['ALPHA_Buy_price'].empty or (execution_df_p['ALPHA_Buy_price'].iloc[0] != execution_df['ALPHA_Buy_price'].iloc[0]) :
                        execution_df = pd.concat([ execution_df_p ,execution_df.loc[:]]).reset_index(drop=True)
                        if execution_df.shape[0] >= 401:
                            execution_df = execution_df.iloc[:400]
                else:
                    if tree_check_decision < 0 and not positve_position:
                        #take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
                        thread_neg = threading.Thread(target=take_full_positive_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in], dic_price[hedge_by]))
                        thread_neg.start()
                        if negative_position:
                            current_position = False
                            take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
                        else:
                            current_position = True
                        positve_position = True
                        negative_position = False
                    elif tree_check_decision > 0 and not negative_position:
                        dic_price = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}

                        thread_pos = threading.Thread(target = take_full_negative_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in]-1, dic_price[hedge_by]))
                        thread_pos.start()
                        if positve_position:
                            current_position = False
                            thread_pos = threading.Thread(target = take_full_negative_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in]-1, dic_price[hedge_by]))
                            thread_pos.start() 
                        else:
                            current_position = True    
                        positve_position = False
                        negative_position = True    
                    execution_df_p = pd.DataFrame({'tick': current_tick,'ALPHA_Buy_price':dic_prcie['ALPHA'],'GAMMA_Buy_price': dic_prcie['GAMMA'], 'THETA_Buy_price': dic_prcie['THETA'],'Buy_decision': take_full_in, 'Sell_decision':hedge_by,'positive_position':positve_position,'Negative_position':negative_position,'Position':current_position   ,'ALPHA - Beta': CAPM_vals['Beta - ALPHA'], 'GAMMA - Beta': CAPM_vals['Beta - GAMMA'], 'THETA - Beta': CAPM_vals['Beta - THETA']})
                    if execution_df['ALPHA_Buy_price'].empty or (execution_df_p['ALPHA_Buy_price'].iloc[0] != execution_df['ALPHA_Buy_price'].iloc[0]) :
                        execution_df = pd.concat([ execution_df_p ,execution_df.loc[:]]).reset_index(drop=True)
                        if execution_df.shape[0] >= 401:
                            execution_df = execution_df.iloc[:400]            
                #execution_df.to_csv(f'execution_df_9.csv', index=False)                    
                     
            # if CAPM_vals['tick_forward']- current_tick  <= -10 and CAPM_vals['tick_forward']- current_tick >= -15:
            #     if positve_position:
            #         my_position = 'Positive'
            #     else:
            #         my_position = 'Negative'
            #     # get_out_gracefully(take_full_in, hedge_by, buy_price, my_position)
            #     thread_get_out = threading.Thread(target = get_out_gracefully, args=(take_full_in, hedge_by, buy_price, my_position))
            #     thread_get_out.start()
                        
                        
            if CAPM_vals['tick_forward']- current_tick  <= -10 and CAPM_vals['tick_forward']- current_tick >= -15:  #and CAPM_vals['tick_forward']- current_tick >= -4
            #     if dic_price[take_full_in] > price_current[take_full_in]:
            #         profitable = False
            #     else:
            #         profitable = True    
            #     if profitable and positve_position:
            #         current_position = False   
            #         positve_position = False
            #         negative_position = False            
            #         resp = session.get('http://localhost:9999/v1/limits')
            #         if resp.ok:
            #             case = resp.json()
            #             if case[0]['gross'] != 0:
            #                 get_out_of_position()
            #     if not profitable and (not positve_position):
                current_position = False   
                positve_position = False
                negative_position = False            
                resp = session.get('http://localhost:9999/v1/limits')
                if resp.ok:
                    case = resp.json()
                    if case[0]['gross'] != 0:
                        get_out_of_position()
            #     else:
            #         pass
            if CAPM_vals['tick_forward']- current_tick  <= -21:
                session.post('http://localhost:9999/v1//commands/cancel?all=1')
                current_position = False   
                positve_position = False
                negative_position = False            
                resp = session.get('http://localhost:9999/v1/limits')
                if resp.ok:
                    case = resp.json()
                    if case[0]['gross'] != 0:
                        get_out_of_position()
                    
                           
                                
            if CAPM_vals['tick_forward']- current_tick > 11 or CAPM_vals['tick_forward']- current_tick<0:
                max_arr = [None]*15
                min_arr = [None]*15           
                           
            if CAPM_vals['tick_forward']- current_tick  == -5:
                session.post('http://localhost:9999/v1//commands/cancel?all=1')
            # Uncomment this string to enable Buy/Sell
            # buy_or_sell(session, expected_return)
            
            #print statement (print, expected_return function, any of the tickers, or CAPM_vals dictionary)
            # print(expected_return)    

In [28]:
if __name__ == '__main__':
    main()

ApiException: fail - cant get tick

In [316]:
def collect_df_for_analysis(count):
    for a in range(20, count+20, 1):
        
        status  = session.get("http://localhost:9999/v1/case")
        status = status.json()
        current_status = status['status']
        if current_status == "ACTIVE":
            sleep(2)
            main(a)
        else:
            sleep(60)    

In [62]:
collect_df_for_analysis(12)

c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\

In [ ]:
session.headers.update(API_KEY)

In [23]:
import requests
import concurrent.futures

 

In [ ]:
resp = session.get('http://localhost:9999/v1/news')
if resp.ok:
    case = resp.json()
    print(case[1])

{'ticker': 'ALPHA', 'type': 'STOCK', 'size': 1, 'position': 60000.0, 'vwap': 176.9838, 'nlv': 9747600.0, 'last': 162.76, 'bid': 162.46, 'bid_size': 100000000.0, 'ask': 162.56, 'ask_size': 1300.0, 'volume': 6702600.0, 'unrealized': -871428.0, 'realized': 112769.0, 'currency': '', 'total_volume': 6702600.0, 'limits': [{'name': 'LIMIT-STOCK', 'units': 1.0}], 'interest_rate': 0.0, 'is_tradeable': True, 'is_shortable': True, 'start_period': 1, 'stop_period': 1, 'description': 'Stock', 'unit_multiplier': 1, 'display_unit': '', 'start_price': 153.381020709021, 'min_price': 20.0, 'max_price': 800.0, 'quoted_decimals': 2, 'trading_fee': 0.02, 'limit_order_rebate': 0.0, 'min_trade_size': 0, 'max_trade_size': 10000, 'required_tickers': None, 'underlying_tickers': None, 'bond_coupon': 0.0, 'interest_payments_per_period': 0, 'base_security': '', 'fixing_ticker': None, 'api_orders_per_second': 10, 'execution_delay_ms': 0, 'interest_rate_ticker': None, 'otc_price_range': 0.0}


In [ ]:
take_full_negative(session,'ALPHA','THETA')

In [ ]:
def get_out()

In [ ]:
#Calculate from capm Max(Mod(Total Average of CAPM - Last 20 Values of CAPM )*Current Price)

In [331]:
# def Take_full_negative(session,position_ticker,hedge_ticker):
for a in range(12):
    resp = session.get('http://localhost:9999/v1/limits')
    if resp.ok:
        case = resp.json()
    if case[0]['net'] <= 100000:
        # store = session.post('http://localhost:9999/v1/orders', params = {'ticker': 'ALPHA', 'type': 'MARKET', 'quantity': 10000, 'action': 'SELL'})
        store1 = session.post('http://localhost:9999/v1/orders', params = {'ticker': 'ALPHA', 'type': 'MARKET', 'quantity': 10000, 'action': 'BUY'})
        # session.post('http://localhost:9999/v1/orders', params = {'ticker': position_ticker, 'type': 'MARKET', 'quantity': 7500, 'action': 'SELL'})

In [333]:
print(store1.json())

{'order_id': 67147, 'period': 1, 'tick': 429, 'trader_id': 'UOFC-1', 'ticker': 'ALPHA', 'quantity': 10000.0, 'price': None, 'type': 'MARKET', 'action': 'BUY', 'quantity_filled': 10000.0, 'vwap': 161.841, 'status': 'TRANSACTED'}


In [ ]:
news = session.get('http://localhost:9999/v1/news')    
if news.ok:
    newsbook = news.json()
    try:
        for i in range(len(newsbook[-1]['body'])):
            if newsbook[-1]['body'][i] == '%':
                CAPM_vals['%Rf'] = round(float(newsbook[-1]['body'][i - 4:i])/100, 4)               
        latest_news = newsbook[0]
        if len(newsbook) > 1:
            for j in range(len(latest_news['body']) - 1, 1, -1):    
                while latest_news['body'][j] != '$':
                    j -= 1
            CAPM_vals['forward'] = float(latest_news['body'][j + 1:-1])
            CAPM_vals['tick_forward'] = int(latest_news['body'][latest_news['body'].index('tick') + 5:latest_news['body'].index(',')])
        return CAPM_vals, len(newsbook)
    except:
        return CAPM_vals, len(newsbook)
raise ApiException('timeout')

In [31]:
news = session.get('http://localhost:9999/v1/news')

In [32]:
print(news.json())

[{'news_id': 13, 'period': 1, 'tick': 566, 'ticker': '', 'headline': 'Private Information 12', 'body': 'Your analysts informed that at tick 577,  the market price of RITM is forecasted to be $4884.15.'}, {'news_id': 12, 'period': 1, 'tick': 523, 'ticker': '', 'headline': 'Private Information 11', 'body': 'Your analysts informed that at tick 534,  the market price of RITM is forecasted to be $4832.15.'}, {'news_id': 11, 'period': 1, 'tick': 464, 'ticker': '', 'headline': 'Private Information 10', 'body': 'Your analysts informed that at tick 475,  the market price of RITM is forecasted to be $4772.62.'}, {'news_id': 10, 'period': 1, 'tick': 417, 'ticker': '', 'headline': 'Private Information 9', 'body': 'Your analysts informed that at tick 428,  the market price of RITM is forecasted to be $4658.02.'}, {'news_id': 9, 'period': 1, 'tick': 369, 'ticker': '', 'headline': 'Private Information 8', 'body': 'Your analysts informed that at tick 380,  the market price of RITM is forecasted to be 

In [100]:
def get_free_cash_beta_start():
    news = session.get('http://localhost:9999/v1/news')
    news = news.json()[-1]
    news_body = news['body']
    news_arr = news_body.split()
    keywords = ['ALPHA', 'THETA', 'GAMMA', '%' ]
    dic = {}
    count = 0
    for item in news_arr:
        for keyword in keywords:
            if keyword in item:
                if keyword in ('ALPHA', 'THETA', 'GAMMA'):
                    dic[keyword] = float(news_arr[count+1])
                else:
                    dic['cash_free_rate'] = float(item[:-2])/100                    
        count +=1                 
    return dic

In [71]:
def news_praser():
    news = session.get('http://localhost:9999/v1/news')
    leng = len(news.json())
    news = news.json()[0]
    news_body = news['body']
    if len(news) > 1:
        news_arr = news_body.split()
        keywords = ['tick', '$' ]
        dic = {}
        count = 0
        for item in news_arr:
            for keyword in keywords:
                if keyword in item:
                    if keyword in ('tick'):
                        dic[keyword] = int(news_arr[count+1][:-1])
                    else:
                        dic['forward_ritm'] = float(item[1:-1])                    
            count +=1                 
        return dic, leng
    return {'tick':0,'price':0}, 1

In [73]:
print(get_free_cash_beta_start())

({'cash_free_rate': 2.79, 'ALPHA': 0.83, 'GAMMA': 2.31, 'THETA': 1.2}, 1)


In [53]:
print(news_praser())

({'tick': 289, 'forward_ritm': 4119.71}, 7)


In [79]:
def main_retry():
    with requests.Session() as session:
        session.headers.update(API_KEY)
        ritm = pd.DataFrame(columns= ['tick','RITM','BID', 'ASK', 'LAST', '%Rm'])
        alpha = pd.DataFrame(columns= ['tick','ALPHA','BID', 'ASK', 'LAST', '%Ri', '%Rm'])
        gamma = pd.DataFrame(columns= ['tick','GAMMA','BID', 'ASK', 'LAST', '%Ri', '%Rm'])
        theta = pd.DataFrame(columns= ['tick','THETA','BID', 'ASK', 'LAST', '%Ri', '%Rm'])
        beta_df = pd.DataFrame(columns= ['tick','ALPHA', 'GAMMA', 'THETA'])
        # try:
        #     capm_df = pd.read_csv('capm_df.csv')
        # except FileNotFoundError:
        capm_df = pd.DataFrame(columns=['ALPHA', 'GAMMA', 'THETA'])
        max_arr = [None]*15
        min_arr = [None]*15
        index_arr = 0
        current_position = False
        positve_position = False
        negative_position = False
        current_tick = get_tick(session)
        
        execution_df = pd.DataFrame(columns = ['ALPHA_Buy_price','GAMMA_Buy_price', 'THETA_Buy_price', 'Buy_decision', 'Sell_decision','positive_position','Negative_position','Position','ALPHA - Beta', 'GAMMA - Beta', 'THETA - Beta'])            
    
        while get_tick(session) < 600 and not shutdown:
            #update the forward market price and rf rate
            temp ,news_len = get_news(session)
            current_tick = get_tick(session)
            
            ##update RITM bid-ask dataframe
            pdt_RITM = pd.DataFrame(pop_prices(session)[0])
            ritmp = pd.DataFrame({'tick':current_tick,'RITM': '', 'BID': pdt_RITM['bid'],'ASK': pdt_RITM['ask'], 'LAST': pdt_RITM['last'], '%Rm': ''})
            if ritm['BID'].empty or ritmp['tick'].iloc[0] != ritm['tick'].iloc[0]:
                ritm = pd.concat([ritmp, ritm.loc[:]]).reset_index(drop=True)
                ritm['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
                if ritm.shape[0] >= 51:
                    ritm = ritm.iloc[:50]
            print(ritm['LAST'].shift(-1))        
            print(ritm['LAST'])
            #generate expected market return paramter
            # if 'forward' in CAPM_vals.keys():
            #     CAPM_vals['%RM'] = (CAPM_vals['forward']-ritm['LAST'].iloc[0])/ritm['LAST'].iloc[0]
            # else:
            #     CAPM_vals['%RM'] = ''
                
                
              
            # ##update ALPHA bid-ask dataframe
            # pdt_ALPHA = pd.DataFrame(pop_prices(session)[1])
            # alphap = pd.DataFrame({'tick':current_tick, 'ALPHA': '', 'BID': pdt_ALPHA['bid'],'ASK': pdt_ALPHA['ask'], 'LAST': pdt_ALPHA['last'], '%Ri': '', '%Rm': ''})
            # if alpha['BID'].empty or alphap['tick'].iloc[0] != alpha['tick'].iloc[0]:
            #     alpha = pd.concat([alphap, alpha.loc[:]]).reset_index(drop=True)
            #     alpha['%Ri'] = (alpha['LAST']/alpha['LAST'].shift(-1)) - 1
            #     alpha['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
            #     if alpha.shape[0] >= 51:
            #         alpha = alpha.iloc[:50]
              
            # ##update GAMMA bid-ask dataframe
            # pdt_GAMMA = pd.DataFrame(pop_prices(session)[2])
            # gammap = pd.DataFrame({'tick':current_tick, 'GAMMA': '', 'BID': pdt_GAMMA['bid'],'ASK': pdt_GAMMA['ask'], 'LAST': pdt_GAMMA['last'], '%Ri': '', '%Rm': ''})
            # if gamma['BID'].empty or gammap['tick'].iloc[0] != gamma['tick'].iloc[0]:
            #     gamma = pd.concat([gammap, gamma.loc[:]]).reset_index(drop=True)
            #     gamma['%Ri'] = (gamma['LAST']/gamma['LAST'].shift(-1)) - 1
            #     gamma['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
            #     if gamma.shape[0] >= 51:
            #         gamma = gamma.iloc[:50]            
            
            # ##update THETA bid-ask dataframe
            # pdt_THETA = pd.DataFrame(pop_prices(session)[3])
            # thetap = pd.DataFrame({'tick':current_tick, 'THETA': '', 'BID': pdt_THETA['bid'],'ASK': pdt_THETA['ask'], 'LAST': pdt_THETA['last'], '%Ri': '', '%Rm': ''})
            # if theta['BID'].empty or thetap['tick'].iloc[0] != theta['tick'].iloc[0]:     
            #     theta = pd.concat([thetap, theta.loc[:]]).reset_index(drop=True)
            #     theta['%Ri'] = (theta['LAST']/theta['LAST'].shift(-1)) - 1
            #     theta['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
            #     if theta.shape[0] >= 51:
            #         theta = theta.iloc[:50]


            # # Step 1: Join the two DataFrames on the common keys
            # # print(alpha)        
            # # print(theta)
            # # print(gamma)
            
                  
            
            # beta_alpha = (alpha['%Ri'].cov(ritm['%Rm']))/(ritm['%Rm'].var())
            # beta_gamma = (gamma['%Ri'].cov(ritm['%Rm']))/(ritm['%Rm'].var())
            # beta_theta = (theta['%Ri'].cov(ritm['%Rm']))/(ritm['%Rm'].var())

            # # print(beta_alpha)
            # # print(beta_gamma)
            # # print(beta_theta)
            
            # beta_p = pd.DataFrame({'ALPHA': beta_alpha, 'GAMMA': beta_gamma, 'THETA': beta_theta}, index = [0])
            # if beta_df['ALPHA'].empty or beta_p['ALPHA'].iloc[0] != beta_df['ALPHA'].iloc[0]:     
            #     beta_df = pd.concat([beta_p, beta_df.loc[:]]).reset_index(drop=True)
            #     if beta_df.shape[0] >= 41:
            #         beta_df = beta_df.iloc[:40]
             
            
            # CAPM_vals['Beta - ALPHA'] = beta_alpha
            # CAPM_vals['Beta - GAMMA'] = beta_gamma
            # CAPM_vals['Beta - THETA'] = beta_theta
            
            # if CAPM_vals['%RM'] != '':
            #     er_alpha = CAPM_vals['%Rf'] + CAPM_vals['Beta - ALPHA'] * (CAPM_vals['%RM'] - CAPM_vals['%Rf'])
            #     er_gamma = CAPM_vals['%Rf'] + CAPM_vals['Beta - GAMMA'] * (CAPM_vals['%RM'] - CAPM_vals['%Rf'])
            #     er_theta = CAPM_vals['%Rf'] + CAPM_vals['Beta - THETA'] * (CAPM_vals['%RM'] - CAPM_vals['%Rf'])
            # else:
            #     er_alpha = 'Wait for market forward price'
            #     er_gamma = 'Wait for market forward price'
            #     er_theta = 'Wait for market forward price'
                
            # expected_return['ALPHA'] = er_alpha
            # expected_return['GAMMA'] = er_gamma
            # expected_return['THETA'] = er_theta
            
            
            # current_tick = get_tick(session)
            # # previous_tick = ''
            # # if current_tick != previous_tick:
            # try:
            #     news = session.get(f"http://localhost:9999/v1/news")
            #     news = news.json()[0]['body']
            # except:
            #     news = "Not able to read news or no news yet."    
            # capm_df_p = pd.DataFrame({'Tick':current_tick,'ALPHA': er_alpha, 'GAMMA': er_gamma,'THETA': er_theta, 'Price-ALPHA':pdt_ALPHA['last'], 'Price-GAMMA':pdt_GAMMA['last'],'Price-THETA':pdt_THETA['last'], 'Price-RITM':pdt_RITM['last'], 'Latest-News': news}, index = [0])
            # if capm_df['ALPHA'].empty or (capm_df_p['GAMMA'].iloc[0] != capm_df['GAMMA'].iloc[0]) :
            #     capm_df = pd.concat([ capm_df_p ,capm_df.loc[:]]).reset_index(drop=True)
            #     if capm_df.shape[0] >= 4001:
            #         capm_df = capm_df.iloc[:4000]
            # # previous_tick = current_tick       
                
            # #capm_df.to_csv(f'capm_df_with_execution_9.csv', index=False)
                
            
            # max_arr[index_arr], min_arr[index_arr], expected_return_max, expected_return_min = calculate_max_min(capm_df)
            
            # index_arr += 1
            
            # if index_arr == 5:
            #     index_arr = 0
            
            # print(CAPM_vals['tick_forward']- current_tick)
            # print(max_arr,min_arr)
            
            # # if None not in max_arr and None not in min_arr and (CAPM_vals['tick_forward']- current_tick == 7) and news_len > 1:
            # #     dic = {'ALPHA':er_alpha,'GAMMA': er_gamma, 'THETA':er_theta}
            # #     max_value = max(dic, key=dic.get)
            # #     min_value = min(dic, key=dic.get)
            # #     if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0:
            # #         take_full_positive(session, max_value, 'THETA')
            # #     else:
            # #         take_full_negative(session, min_value, 'THETA')    
                    
            # if (CAPM_vals['tick_forward']- current_tick == 9):
            #     dic_price = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}
            #     tree_check_decision = CAPM_vals['forward']-ritm.head(4)['LAST'].mean()
            # else:
            #     price_current = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}    
            
            # if None not in max_arr and None not in min_arr and (CAPM_vals['tick_forward']- current_tick == 8) and news_len > 1:
            #     take_full_in = max(max_arr)
            #     beta_full_in = beta_df.head(4)[take_full_in].mean()
            #     # if beta_full_in < 0:
            #     #     opposite_position = True
            #     # else:
            #     opposite_position = False    
                
            #     # if take_full_in:
            #     #     if CAPM_vals['Beta - ALPHA'] > CAPM_vals['Beta - GAMMA']:
            #     #         if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0:
            #     #             take_full_in = 'ALPHA'
            #     #         else:
            #     #             take_full_in = 'GAMMA'    
            #     #     else:
            #     #         if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0:
            #     #             take_full_in = 'GAMMA'
            #     #         else:
            #     #             take_full_in = 'ALPHA'        
                             
                                
            #     hedge_by =  max(min_arr)
            #     dic_beta = {'ALPHA': CAPM_vals['Beta - ALPHA'], 'GAMMA': CAPM_vals['Beta - GAMMA'], 'THETA': CAPM_vals['Beta - THETA']}
                
            #     dic_prcie = dic_price
                
            #     print(take_full_in)
            #     print(hedge_by)
            #     # if abs(CAPM_vals['forward']-ritm['LAST'].iloc[0])>70:
            #     # if CAPM_vals['tick_forward']- current_tick < 2 and not current_position:
            #     #     if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0 and not positve_position:
            #     #         take_full_positive(session, take_full_in, hedge_by)
            #     # elif CAPM_vals['tick_forward']- current_tick < 0 and current_position:
            #     #     if positve_position:
            #     #         take_full_negative(session, take_full_in, hedge_by)
            #     #     else:
            #     #         take_full_positive(session, take_full_in, hedge_by)                
            #     # elif CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0 and not positve_position:
            #     #     take_full_positive(session, take_full_in, hedge_by)
            #     #     if negative_position:
            #     #         current_position = False
            #     #         take_full_positive(session, take_full_in, hedge_by)
            #     #     else:
            #     #         current_position = True
            #     #     positve_position = True
            #     #     negative_position = False
            #     # elif CAPM_vals['forward']-ritm['LAST'].iloc[0] < 0 and not negative_position:
            #     #     take_full_negative(session, take_full_in, hedge_by)
            #     #     if positve_position:
            #     #         current_position = False
            #     #         take_full_negative(session, take_full_in, hedge_by) 
            #     #     else:
            #     #         current_position = True    
            #     #     positve_position = False
            #     #     negative_position = True
            # # else:
            #     # if CAPM_vals['tick_forward']- current_tick < 2 and not current_position:
            #     #     if CAPM_vals['forward']-ritm['LAST'].iloc[0] > 0 and not positve_position:
            #     #         take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
            #     # elif CAPM_vals['tick_forward']- current_tick < 0 and current_position:
            #     #     if positve_position:
            #     #         take_full_negative_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
            #     #     else:
            #     #         take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by]) 
            #     buy_price = dic_price[take_full_in]
            #     if not opposite_position:              
            #         if tree_check_decision > 0 and not positve_position:
            #             #take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
            #             thread_neg = threading.Thread(target=take_full_positive_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in], dic_price[hedge_by]))
            #             thread_neg.start()
            #             if negative_position:
            #                 current_position = False
            #                 take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
            #             else:
            #                 current_position = True
            #             positve_position = True
            #             negative_position = False
            #         elif tree_check_decision < 0 and not negative_position:
            #             dic_price = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}
            #             thread_pos = threading.Thread(target = take_full_negative, args=(session, take_full_in, hedge_by))
            #             thread_pos.start()
            #             if positve_position:
            #                 current_position = False
            #                 thread_pos = threading.Thread(target = take_full_negative, args=(session, take_full_in, hedge_by))
            #                 thread_pos.start() 
            #             else:
            #                 current_position = True    
            #             positve_position = False
            #             negative_position = True    
            #         execution_df_p = pd.DataFrame({'tick': current_tick,'ALPHA_Buy_price':dic_prcie['ALPHA'],'GAMMA_Buy_price': dic_prcie['GAMMA'], 'THETA_Buy_price': dic_prcie['THETA'],'Buy_decision': take_full_in, 'Sell_decision':hedge_by,'positive_position':positve_position,'Negative_position':negative_position,'Position':current_position   ,'ALPHA - Beta': CAPM_vals['Beta - ALPHA'], 'GAMMA - Beta': CAPM_vals['Beta - GAMMA'], 'THETA - Beta': CAPM_vals['Beta - THETA']})
            #         if execution_df['ALPHA_Buy_price'].empty or (execution_df_p['ALPHA_Buy_price'].iloc[0] != execution_df['ALPHA_Buy_price'].iloc[0]) :
            #             execution_df = pd.concat([ execution_df_p ,execution_df.loc[:]]).reset_index(drop=True)
            #             if execution_df.shape[0] >= 401:
            #                 execution_df = execution_df.iloc[:400]
            #     else:
            #         if tree_check_decision < 0 and not positve_position:
            #             #take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
            #             thread_neg = threading.Thread(target=take_full_positive_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in], dic_price[hedge_by]))
            #             thread_neg.start()
            #             if negative_position:
            #                 current_position = False
            #                 take_full_positive_limit(session, take_full_in, hedge_by, dic_prcie[take_full_in], dic_prcie[hedge_by])
            #             else:
            #                 current_position = True
            #             positve_position = True
            #             negative_position = False
            #         elif tree_check_decision > 0 and not negative_position:
            #             dic_price = {'ALPHA': pdt_ALPHA['last'], 'GAMMA': pdt_GAMMA['last'], 'THETA': pdt_THETA['last']}

            #             thread_pos = threading.Thread(target = take_full_negative_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in]-1, dic_price[hedge_by]))
            #             thread_pos.start()
            #             if positve_position:
            #                 current_position = False
            #                 thread_pos = threading.Thread(target = take_full_negative_limit, args=(session, take_full_in, hedge_by, dic_price[take_full_in]-1, dic_price[hedge_by]))
            #                 thread_pos.start() 
            #             else:
            #                 current_position = True    
            #             positve_position = False
            #             negative_position = True    
            #         execution_df_p = pd.DataFrame({'tick': current_tick,'ALPHA_Buy_price':dic_prcie['ALPHA'],'GAMMA_Buy_price': dic_prcie['GAMMA'], 'THETA_Buy_price': dic_prcie['THETA'],'Buy_decision': take_full_in, 'Sell_decision':hedge_by,'positive_position':positve_position,'Negative_position':negative_position,'Position':current_position   ,'ALPHA - Beta': CAPM_vals['Beta - ALPHA'], 'GAMMA - Beta': CAPM_vals['Beta - GAMMA'], 'THETA - Beta': CAPM_vals['Beta - THETA']})
            #         if execution_df['ALPHA_Buy_price'].empty or (execution_df_p['ALPHA_Buy_price'].iloc[0] != execution_df['ALPHA_Buy_price'].iloc[0]) :
            #             execution_df = pd.concat([ execution_df_p ,execution_df.loc[:]]).reset_index(drop=True)
            #             if execution_df.shape[0] >= 401:
            #                 execution_df = execution_df.iloc[:400]            
            #     #execution_df.to_csv(f'execution_df_9.csv', index=False)                    
                     
            # # if CAPM_vals['tick_forward']- current_tick  <= -10 and CAPM_vals['tick_forward']- current_tick >= -15:
            # #     if positve_position:
            # #         my_position = 'Positive'
            # #     else:
            # #         my_position = 'Negative'
            # #     # get_out_gracefully(take_full_in, hedge_by, buy_price, my_position)
            # #     thread_get_out = threading.Thread(target = get_out_gracefully, args=(take_full_in, hedge_by, buy_price, my_position))
            # #     thread_get_out.start()
                        
                        
            # if CAPM_vals['tick_forward']- current_tick  <= -10 and CAPM_vals['tick_forward']- current_tick >= -15:  #and CAPM_vals['tick_forward']- current_tick >= -4
            # #     if dic_price[take_full_in] > price_current[take_full_in]:
            # #         profitable = False
            # #     else:
            # #         profitable = True    
            # #     if profitable and positve_position:
            # #         current_position = False   
            # #         positve_position = False
            # #         negative_position = False            
            # #         resp = session.get('http://localhost:9999/v1/limits')
            # #         if resp.ok:
            # #             case = resp.json()
            # #             if case[0]['gross'] != 0:
            # #                 get_out_of_position()
            # #     if not profitable and (not positve_position):
            #     current_position = False   
            #     positve_position = False
            #     negative_position = False            
            #     resp = session.get('http://localhost:9999/v1/limits')
            #     if resp.ok:
            #         case = resp.json()
            #         if case[0]['gross'] != 0:
            #             get_out_of_position()
            # #     else:
            # #         pass
            # if CAPM_vals['tick_forward']- current_tick  <= -21:
            #     session.post('http://localhost:9999/v1//commands/cancel?all=1')
            #     current_position = False   
            #     positve_position = False
            #     negative_position = False            
            #     resp = session.get('http://localhost:9999/v1/limits')
            #     if resp.ok:
            #         case = resp.json()
            #         if case[0]['gross'] != 0:
            #             get_out_of_position()
                    
                           
                                
            # if CAPM_vals['tick_forward']- current_tick > 11 or CAPM_vals['tick_forward']- current_tick<0:
            #     max_arr = [None]*5
            #     min_arr = [None]*5           
                           
            # if CAPM_vals['tick_forward']- current_tick  == -5:
            #     session.post('http://localhost:9999/v1//commands/cancel?all=1')
            # # Uncomment this string to enable Buy/Sell
            # # buy_or_sell(session, expected_return)
            
            # #print statement (print, expected_return function, any of the tickers, or CAPM_vals dictionary)
            # # print(expected_return)    

In [ ]:
def get_prices():
    

In [81]:
print(pop_prices(session))

[{'ticker': 'RITM', 'type': 'INDEX', 'size': 1, 'position': 0.0, 'vwap': 0.0, 'nlv': 0.0, 'last': 4559.43, 'bid': 4559.08, 'bid_size': 10000.0, 'ask': 4559.78, 'ask_size': 10000.0, 'volume': 0.0, 'unrealized': 0.0, 'realized': 0.0, 'currency': '', 'total_volume': 0.0, 'limits': [{'name': 'LIMIT-STOCK', 'units': 1.0}], 'interest_rate': 0.0, 'is_tradeable': False, 'is_shortable': False, 'start_period': 1, 'stop_period': 1, 'description': 'Market Index', 'unit_multiplier': 1, 'display_unit': 'index unit', 'start_price': 4475.27457415471, 'min_price': 100.0, 'max_price': 10000.0, 'quoted_decimals': 2, 'trading_fee': 0.0, 'limit_order_rebate': 0.0, 'min_trade_size': 0, 'max_trade_size': 1, 'required_tickers': None, 'underlying_tickers': None, 'bond_coupon': 0.0, 'interest_payments_per_period': 0, 'base_security': '', 'fixing_ticker': None, 'api_orders_per_second': 10, 'execution_delay_ms': 0, 'interest_rate_ticker': None, 'otc_price_range': 0.0}, {'ticker': 'ALPHA', 'type': 'STOCK', 'size':

In [171]:
def main_check():
    # with requests.Session() as session:
    current_tick = get_tick(session)
    df_price = pd.DataFrame(columns= ['tick','ALPHA','GAMMA', 'THETA', 'RITM'])
    df_decision = pd.DataFrame(columns=['tick','Ri_ALPHA','Ri_GAMMA', 'Ri_THETA','Rm'])
    # beta_df = pd.DataFrame(columns= ['ALPHA', 'GAMMA', 'THETA'])
    cash_free_initial_beta = None
    cash_free_rate = None
    count_table = 30
    first_position = True
    news_price = {'tick':0,'price':0}
    position_type = None
    my_ticker = None
    is_profit = False

    while get_tick(session) < 600 and not shutdown:
        current_tick = get_tick(session)
        if current_tick >= 1:
            news_price, leng = news_praser()
            if cash_free_initial_beta is None:
                cash_free_initial_beta = get_free_cash_beta_start()
                cash_free_rate = cash_free_initial_beta['cash_free_rate']
                print(cash_free_initial_beta)
                

            if leng == 2 and first_position:
                prices = pop_prices(session)
                temp_beta_alpha = cash_free_initial_beta['ALPHA']
                temp_beta_gamma = cash_free_initial_beta['GAMMA']
                temp_beta_theta = cash_free_initial_beta['THETA']
                
                temp_forward = news_price['forward_ritm']
                capm_alpha_price = (cash_free_rate + (temp_beta_alpha*(((temp_forward)/prices['RITM']-1)-cash_free_rate)))*prices['ALPHA']
                capm_gamma_price = (cash_free_rate + (temp_beta_gamma*(((temp_forward)/prices['RITM']-1)-cash_free_rate)))*prices['GAMMA']
                capm_theta_price =(cash_free_rate + (temp_beta_theta*(((temp_forward)/prices['RITM']-1)-cash_free_rate)))*prices['THETA']
                arr_beta = [temp_beta_alpha,temp_beta_gamma, temp_beta_theta]
                decision_dic = {'ALPHA':capm_alpha_price,'GAMMA':capm_gamma_price,'THETA':capm_theta_price}
                decision_dic_abs = {'ALPHA':abs(capm_alpha_price),'GAMMA':abs(capm_gamma_price),'THETA':abs(capm_theta_price)}

                print("temp_forward-prices['RITM']",temp_forward-prices['RITM'])
                print("cash_free_rate",cash_free_rate)
                print("prices['RITM']",prices['RITM'])
                print("(temp_forward-prices['RITM'])/prices['RITM']",(temp_forward-prices['RITM'])/prices['RITM'])
                print("temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate):",temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate))
                print("(cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate)))", (cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate))))
                print("(cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate)))*prices['ALPHA']", cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate))*prices['ALPHA'])
                
                print(decision_dic)
                print(decision_dic_abs)
                full_in_abs = max(decision_dic_abs, key=decision_dic_abs.get)
                if decision_dic[full_in_abs] == -decision_dic_abs[full_in_abs]:
                    hedge_by = max(decision_dic, key=decision_dic.get)
                    full_in = full_in_abs
                else:
                    full_in = full_in_abs
                    hedge_by = min(decision_dic, key=decision_dic.get) 
                    
                if decision_dic[full_in] > -0.1:
                    thread_neg = threading.Thread(target=take_full_positive_limit, args=(session, full_in, hedge_by, prices[full_in], prices[hedge_by]))
                    thread_neg.start()
                    position_type = 'positive'
                    buy_price, my_ticker  = prices[full_in], full_in
                else:
                    thread_neg = threading.Thread(target=take_full_negative, args=(session, full_in, hedge_by))
                    thread_neg.start()
                    position_type = 'negative'
                    buy_price, my_ticker  = prices[full_in], full_in
                        
                first_position = False

                    
                    
                    
                    

            prices = pop_prices(session)
            df_price_p = pd.DataFrame({ 'tick': current_tick,'ALPHA': prices['ALPHA'],'GAMMA': prices['GAMMA'], 'THETA': prices['THETA'], 'RITM': prices['RITM']}, index = [0])
            if df_price['ALPHA'].empty or df_price_p['tick'].iloc[0] != df_price['tick'].iloc[0]:
                df_price = pd.concat([df_price_p, df_price.loc[:]]).reset_index(drop=True)
                # ritm['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
                if df_price.shape[0] >= count_table + 1:
                    df_price = df_price.iloc[:count_table]


            Ri_ALPHA = (df_price['ALPHA']/df_price['ALPHA'].shift(-1)) - 1
            Ri_GAMMA = (df_price['GAMMA']/df_price['GAMMA'].shift(-1)) - 1
            Ri_THETA = (df_price['THETA']/df_price['THETA'].shift(-1)) - 1
            RM = (df_price['RITM']/df_price['RITM'].shift(-1)) - 1

            df_decision_p = pd.DataFrame({ 'tick': current_tick,'Ri_ALPHA': Ri_ALPHA,'Ri_GAMMA': Ri_GAMMA, 'Ri_THETA': Ri_THETA, 'Rm': RM}, index = [0])
            if df_decision['Ri_ALPHA'].empty or df_decision_p['tick'].iloc[0] != df_decision['tick'].iloc[0]:
                df_decision = pd.concat([df_decision_p, df_decision.loc[:]]).reset_index(drop=True)
                # ritm['%Rm'] = (ritm['LAST']/ritm['LAST'].shift(-1)) - 1
                if df_decision.shape[0] >= count_table + 1:
                    df_decision = df_decision.iloc[:count_table]
                
            beta_alpha = (df_decision['Ri_ALPHA'].cov(df_decision['Rm']))/(df_decision['Rm'].var())
            beta_gamma = (df_decision['Ri_GAMMA'].cov(df_decision['Rm']))/(df_decision['Rm'].var())
            beta_theta = (df_decision['Ri_THETA'].cov(df_decision['Rm']))/(df_decision['Rm'].var())
            # df_decision.to_csv(f'df_decision.csv', index=False)
            # df_price.to_csv(f'df_price.csv', index=False)
                
            # beta_p = pd.DataFrame({'ALPHA': beta_alpha, 'GAMMA': beta_gamma, 'THETA': beta_theta}, index = [0])
            # if beta_df['ALPHA'].empty or beta_p['ALPHA'].iloc[0] != beta_df['ALPHA'].iloc[0]:     
            #     beta_df = pd.concat([beta_p, beta_df.loc[:]]).reset_index(drop=True)
            #     if beta_df.shape[0] >= 41:
            #         beta_df = beta_df.iloc[:40]
            # beta_df.to_csv(f'df_beta.csv', index=False)

            # if leng > 2:
            #     prices = pop_prices(session)
            #     temp_forward = news_price['forward_ritm']
            #     decision_dic = {'ALPHA':(cash_free_rate + (temp_beta_alpha*(((beta_alpha-prices['RITM'])/prices['RITM'])-cash_free_rate)))*prices['ALPHA'],'GAMMA':(cash_free_rate + (temp_beta_gamma*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate)))*prices['GAMMA'],'THETA':(cash_free_rate + (temp_beta_theta*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate)))*prices['THETA']}
            #     full_in = max(decision_dic, key=decision_dic.get)
            #     hedge_by = min(decision_dic, key=decision_dic.get)
            #     if (temp_forward-prices[0]['last']) > 0:
            #         thread_neg = threading.Thread(target=take_full_positive_limit, args=(session, full_in, hedge_by, prices[full_in], prices[hedge_by]))
            #         thread_neg.start()

            # if news_price['forward_ritm']- current_tick  <= -10 and news_price['forward_ritm']- current_tick >= -15:  #and CAPM_vals['tick_forward']- current_tick >= -4
            #     #     if dic_price[take_full_in] > price_current[take_full_in]:
            #     #         profitable = False
            #     #     else:
            #     #         profitable = True    
            #     #     if profitable and positve_position:
            #     #         current_position = False   
            #     #         positve_position = False
            #     #         negative_position = False            
            #     #         resp = session.get('http://localhost:9999/v1/limits')
            #     #         if resp.ok:
            #     #             case = resp.json()
            #     #             if case[0]['gross'] != 0:
            #     #                 get_out_of_position()
            #     #     if not profitable and (not positve_position):           
            #     resp = session.get('http://localhost:9999/v1/limits')
            #     if resp.ok:
            #         case = resp.json()
            #         if case[0]['gross'] != 0:
            #             get_out_of_position()
            if leng > 2 and news_price['tick'] - current_tick == 10 and news_price['tick'] - current_tick  > -500 :
                prices = pop_prices(session)
                temp_beta_alpha = beta_alpha
                temp_beta_gamma = beta_gamma
                temp_beta_theta = beta_theta
                temp_forward = news_price['forward_ritm']
                expected_ret_market = temp_forward/prices['RITM']-1
                print("temp_forward",temp_forward)
                
                
                
                
                print("temp_forward-prices['RITM']",temp_forward-prices['RITM'])
                print("cash_free_rate",cash_free_rate)
                print("prices['RITM']",prices['RITM'])
                print("(temp_forward-prices['RITM'])/prices['RITM']",(temp_forward-prices['RITM'])/prices['RITM'])
                print("temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate):",temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate))
                print("(cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate)))", (cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate))))
                print("(cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate)))*prices['ALPHA']", cash_free_rate + (temp_beta_alpha*(((temp_forward-prices['RITM'])/prices['RITM'])-cash_free_rate))*prices['ALPHA'])
                
                print(prices)
                print(temp_beta_alpha)
                capm_alpha_price = (cash_free_rate + (temp_beta_alpha*(expected_ret_market-cash_free_rate)) )*prices['ALPHA']
                print(capm_alpha_price)
                capm_gamma_price = (cash_free_rate + (temp_beta_gamma*(((temp_forward)/prices['RITM']-1)-cash_free_rate)))*prices['GAMMA']
                capm_theta_price =(cash_free_rate + (temp_beta_theta*(((temp_forward)/prices['RITM']-1)-cash_free_rate)))*prices['THETA']
                arr_beta = [temp_beta_alpha,temp_beta_gamma, temp_beta_theta]
                decision_dic = {'ALPHA':capm_alpha_price,'GAMMA':capm_gamma_price,'THETA':capm_theta_price}
                decision_dic_abs = {'ALPHA':abs(capm_alpha_price),'GAMMA':abs(capm_gamma_price),'THETA':abs(capm_theta_price)}
                full_in_abs = max(decision_dic_abs, key=decision_dic_abs.get)
                print(full_in_abs)
                print(decision_dic)
                print(decision_dic_abs)
                if decision_dic[full_in_abs] == -decision_dic_abs[full_in_abs]:
                    hedge_by = max(decision_dic, key=decision_dic.get)
                    full_in, my_ticker  = full_in_abs, full_in
                else:
                    full_in = full_in_abs
                    hedge_by = min(decision_dic, key=decision_dic.get) 
                    
                if decision_dic[full_in] > 0:
                    thread_neg = threading.Thread(target=take_full_positive_limit, args=(session, full_in, hedge_by, prices[full_in], prices[hedge_by]))
                    thread_neg.start()
                    position_type = 'positive'
                    buy_price, my_ticker  = prices[full_in], full_in
                else:
                    thread_neg = threading.Thread(target=take_full_negative, args=(session, full_in, hedge_by))
                    thread_neg.start()
                    position_type = 'negative'
                    buy_price, my_ticker = prices[full_in], full_in
            prices = pop_prices(session)       
            if position_type == 'positive':
                if buy_price < prices[my_ticker]:
                    is_profit = True
                else:
                    is_profit = False
            elif position_type == 'negative':
                if buy_price > prices[my_ticker]:
                    is_profit = True
                else:
                    is_profit = False
                            
                    
            try:        
                if news_price['tick'] - current_tick  <= -10 and news_price['tick']- current_tick >= -15:  #and CAPM_vals['tick_forward']- current_tick >= -4
                    #     if dic_price[take_full_in] > price_current[take_full_in]:
                    #         profitable = False
                    #     else:
                    #         profitable = True    
                    #     if profitable and positve_position:
                    #         current_position = False   
                    #         positve_position = False
                    #         negative_position = False            
                    #         resp = session.get('http://localhost:9999/v1/limits')
                    #         if resp.ok:
                    #             case = resp.json()
                    #             if case[0]['gross'] != 0:
                    #                 get_out_of_position()
                    #     if not profitable and (not positve_position):
                    current_position = False   
                    positve_position = False
                    negative_position = False            
                    resp = session.get('http://localhost:9999/v1/limits')
                    if resp.ok:
                        case = resp.json()
                        if case[0]['gross'] != 0:
                            get_out_of_position()         
                            
                if news_price['tick'] - current_tick  == -5:
                    session.post('http://localhost:9999/v1//commands/cancel?all=1')            
                
            except:
                pass                                               


            print("**", current_tick)
            print(beta_alpha)
            print(beta_gamma)
            print(beta_theta)


In [174]:
main_check()

{'cash_free_rate': 0.011399999999999999, 'ALPHA': 1.51, 'GAMMA': 2.3, 'THETA': 1.56}
** 12
nan
nan
nan
** 12
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan


c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\

** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan


c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\

** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan


...

** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan


c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\

** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan
** 13
nan
nan
nan


c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\lib\site-packages\pandas\core\nanops.py:1601: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
c:\Users\mayan\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\mayan\anaconda3\

** 13
nan
nan
nan
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14
2.6220736693290516
4.216614908159697
6.491990090913115
** 14


In [80]:
main_retry()

C:\Users\trader13\AppData\Local\Temp\ipykernel_11016\827723016.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ritm = pd.concat([ritmp, ritm.loc[:]]).reset_index(drop=True)


0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype: float64
0    4321.59
Name: LAST, dtype: float64
0   NaN
Name: LAST, dtype

KeyboardInterrupt: 